In [7]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [8]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    x = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in x:
            alt_titles[key] = x[key]
            x.pop(key)
    if len(alt_titles) > 0:
        x['alt_titles'] = alt_titles

    return x

In [16]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)


def etl():
    #Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv('ratings.csv')
    
    #Open the read the Wikipedia data JSON file.
    with open('wikipedia.movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
    
     # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw \
                   if 'No. of episodes' not in movie]
    
    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    cleaned_wiki_movies = [clean_movie(movie) for movie in wiki_movies]
    
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    cleaned_movies_df = pd.DataFrame(cleaned_wiki_movies)
    
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    
    try:
        cleaned_movies_df['imdb_id'] = cleaned_movies_df['imdb_link'].str.extract(r"(tt\d{7})")
        cleaned_movies_df = cleaned_movies_df.sort_values(by=['imdb_id'])
        cleaned_movies_df = cleaned_movies_df.drop_duplicates(subset= ['imdb_id'])
        
    except:
        print('no link')
    
    
    #7. write a list comprehension to keep the columns that have non-null values from the wiki_movies_df DataFrame
    wiki_movies_df = cleaned_movies_df
    null_cols = wiki_movies_df.columns[wiki_movies_df.isnull().all()]
    wiki_movies_df.drop(null_cols, axis = 1, inplace = True)
    
    
    
     # 8. Create a variable that will hold the non-null values from the “Box office” column.     
    box_office = wiki_movies_df['Box office'].dropna()

       
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office[box_office.map(lambda x: type(x) != str)]
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    
    
    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()
    
    
    
   # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
    matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()
    
    test = box_office.str.extract(f'({form_one}|{form_two})')
    

    # 12. Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
    
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    budget[~matches_form_one & ~matches_form_two]
    
    
    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    
    
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()
    running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    
    
    #drop from dataset
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)

    
    # Return three variables. The first is the wiki_movies_df DataFrame
    return wiki_movies_df, kaggle_metadata, ratings 
    
    #display(wiki_movies_df) 
    
    

In [22]:
pd.set_option("display.max_columns", None)


In [23]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 'C://Users/kimberlyrbraud/vdata/mod_8/Mod_8_V2/08-ETL/Resources/'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [24]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = etl()

/Users/kimberlybraud/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:124: FutureWarning: The default value of regex will change from True to False in a future version.


In [25]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [26]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,Narrated by,Music by,Cinematography,Edited by,Productioncompany,Distributed by,Release date,Country,Language,Budget,Written by,Genre,Theme music composer,Country of origin,Original language(s),Producer(s),Editor(s),Production company(s),Original network,Original release,Productioncompanies,Executive producer(s),Production location(s),Distributor,Picture format,Audio format,Voices of,Followed by,Composer(s),Created by,Preceded by,Author,Publisher,Publication date,Media type,Pages,ISBN,OCLC,LC Class,Cover artist,Series,Set in,Adaptation by,Suggested by,alt_titles,Released,Recorded,Venue,Length,Label,Director,Producer,Area,Coordinates,Status,Opening date,Closing date,Replaced,Replaced by,Name,Attraction type,Music,Duration,Animation by,Color process,Characters,Date premiered,Place premiered,Setting,Original language,Subject,Text,Nationality,Portrayed by,Alias,Species,Gender,Family,Alma mater,Screen story by,imdb_id,box_office,release_date,running_time
6838,https://en.wikipedia.org/wiki/I_Don%27t_Feel_a...,2017.0,https://www.imdb.com/title/tt0005710514/,I Don't Feel at Home in This World Anymore,Macon Blair,"[Mette-Marie Kongsved, Neil Kopp, Vincent Savi...",NaN,NaN,NaN,"[Melanie Lynskey, Elijah Wood, David Yow, Jane...",NaN,"[Brooke Blair, Will Blair]",Larkin Seiple,Tomas Vengris,"[Film Science, XYZ Films]",Netflix,"[January 19, 2017, (, 2017-01-19, ), (, Sundan...",United States,English,NaN,Macon Blair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0005710,NaN,2017-01-19,96.0
6848,https://en.wikipedia.org/wiki/Burning_Sands_(2...,2017.0,https://www.imdb.com/title/tt0005826432/,Burning Sands,Gerard McMurray,"[Stephanie Allain, Jason Michael Berman, Regin...",NaN,NaN,NaN,"[Trevor Jackson, Alfre Woodard, Steve Harris, ...",NaN,Kevin Lax,Isiah Donté Lee,Evan Schrodek,"[Mandalay Pictures, Homegrown Pictures, Hudlin...",Netflix,"[January 24, 2017, (, 2017-01-24, ), (, Sundan...",United States,English,NaN,"[Gerard McMurray, Christine Berg]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0005826,NaN,2017-01-24,102.0
6689,/wiki/The_Other_Side_of_the_Door_(2016_film),2016.0,https://www.imdb.com/title/tt0007149/,The Other Side of the Door,Johannes Roberts,"[Alexandre Aja, Rory Aitken, Ben Pugh]",NaN,NaN,NaN,"[Sarah Wayne Callies, Jeremy Sisto]",NaN,Joseph Bishara,Maxime Alexandre,Baxter,NaN,20th Century Fox,"[March 4, 2016, (, 2016-03-04, )]","[United Kingdom, India, United States]",English,$5 million,"[Ernest Riera, Johannes Roberts]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[42, Kriti Productions, TSG Entertainment]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0007149,14300000.0,2016-03-04,95.0
3696,https://en.wikipedia.org/wiki/Lemony_Snicket%2...,2004.0,https://www.imdb.com/title/tt00339291/,Lemony Snicket's A Series of Unfortunate Events,Brad Silberling,"[Laurie MacDonald, Walter F. Parkes, Jim Van W...",Robert Gordon,NaN,"[A Series of Unfortunate Events, by, Lemony Sn...","[Jim Carrey, Liam Aiken, Emily Browning, Timot...",NaN,Thomas Newman,Emmanuel Lubezki,Michael Kahn,"[Paramount Pictures, DreamWorks Pictures, Nick...","[Paramount Pictures, (North America), DreamWor...","December 17, 2004",United States,English,$140-142 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [27]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Country',
 'Language',
 'Budget',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'alt_titles',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Area',
 'Coordinates',
 'Status',
 